## Main


In [ ]:

"""
main.py:
Schedules and publishes Instagram posts from an Excel spreadsheet,
managing post history and retrying failed uploads.
"""

import os
import json
import random
import time
import datetime
import logging
import pprint


import pandas as pd
from pathlib import Path

from instagrapi.exceptions import ClientError


from ig_config import Config
from ig_client import IgClient
from ig_data import IgPostData
from ig_utils import IgUtils, create_post_dataframe, save_post_dataframe, correct_orientation
from ig_post_manager import IgPostManager

In [ ]:
# Configure logging (optional, but recommended)
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Load project configuration file
config_file_path = r"C:\Users\manue\Documents\GitHubMLSN\sn-libraries\notebooks\ig_JK_config.json"  # Replace with the actual path
config = Config(config_file_path=config_file_path)  # Create Config object with path
username = config.get("username")
password = config.get("password")

# Authenticate using IgClient
igcl = IgClient(session_file=config.get("settings_file_path"))  
igcl.login(username, password)

#Instantiate objects
ig_utils = IgUtils(igcl)
post_manager = IgPostManager(igcl)

In [ ]:
# post test

path = r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\IMG_3239.JPG"
correct_orientation(Path(path))

caption = " more by #JKTravels"


In [ ]:

location_comas = ig_utils.get_location_by_pk(250765185)


In [ ]:
# one photo upload_photo

ig_photo_post = post_manager.upload_photo(
    photo_path=path, caption=caption, location=location_comas)


In [ ]:

print(type(ig_photo_post))
print(ig_photo_post.location_name)

In [ ]:
print(ig_photo_post.media_id)
print(ig_photo_post.media_type)
print(ig_photo_post.caption)
print(ig_photo_post.timestamp)
print(ig_photo_post.media_url)

# ... (print other attributes)


In [ ]:
# one video_post with upload_video
location_larcomar = ig_utils.get_location_by_pk(101440375504236)
path = r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\Belmond Miraflores Park.mp4"
caption = "Some tim ago I took this where today sits the wonderfull hotel @belmondmiraflorespark"


In [ ]:
ig_video_post = post_manager.upload_video(video_path=path, caption=caption, location=location_larcomar)

In [ ]:
print(ig_video_post)

In [ ]:
# upload album post with upload_album
media_paths = [
    Path(r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\IMG_3197.JPG"),
    Path(r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\IMG_3227.JPG"),
    Path(r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\VID_20151108_162632485.mp4"),
    Path(r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\IMG_3197.JPG")  # Add more paths as needed
]
for path in media_paths:
    correct_orientation(path)  # Correct orientation before uploading
caption = """
street photography in Lima
"""
ig_album_post = post_manager.upload_album(paths=media_paths, caption=caption, location=location_comas)


In [ ]:

print(ig_album_post)


In [ ]:
# one reel upload_reel 

path = r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\dog.mp4"
caption = "dogs run on further down the Malecom"

ig_reel = post_manager.upload_video(path, caption=caption, location= location_larcomar)

In [ ]:
print(type(ig_reel))
print(ig_reel)

In [ ]:
# one reel with music upload_video_with_music

track_id: str = "318570269071669"

track = IgUtils.get_track_by_id(track_id)
print(type(track))
print(track)


In [ ]:
path = r"C:\Users\manue\Documents\GitHubMLSN\JK Peru Photos\dog.mp4"
caption = "dogs run on further down the Malecom towards Barranco"
reel_with_music = post_manager.upload_reel_with_music(path=path, caption=caption, track=track, location=location_larcomar)